<a href="https://colab.research.google.com/github/venkat2ram/Keras-and-Tensorflow/blob/master/uda_L10_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#############Not able to run program due to RAM issue################

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
url=tf.keras.utils.get_file('song_data.csv','https://drive.google.com/uc?id=1LiJFZd41ofrWoBtW-pMYsfz1w8Ny0Bj8')

65290240/Unknown - 0s 0us/step

In [3]:
dataset=pd.read_csv(url)

In [4]:
dataset=dataset[:1000]

In [5]:
def create_lyrics_corpus(dataset, field):
  # Remove all other punctuation
  dataset[field] = dataset[field].str.replace('[{}]'.format(string.punctuation), '')
  # Make it lowercase
  dataset[field] = dataset[field].str.lower()
  # Make it one long string to split by line
  lyrics = dataset[field].str.cat()
  corpus = lyrics.split('\n')
  # Remove any trailing whitespace
  for l in range(len(corpus)):
    corpus[l] = corpus[l].rstrip()
  # Remove any empty lines
  corpus = [l for l in corpus if l != '']

  return corpus

def tokenize_data(corpus,num_words=-1):
  if num_words>-1:
    tokenizer=Tokenizer(num_words=num_words)
  else:
    tokenizer=Tokenizer()
  tokenizer.fit_on_texts(corpus)
  return tokenizer


In [6]:
corpus=create_lyrics_corpus(dataset,'text')
tokenizer=tokenize_data(corpus)


In [7]:
total_words=len(tokenizer.word_index)+1

In [8]:
token_list=tokenizer.texts_to_sequences(['look at her face its a wonderful face'])[0]

In [9]:
sequences=[]
for line in corpus:
  token_list=tokenizer.texts_to_sequences([line])[0]
  for i in range(1,len(token_list)):
    n_gram_sequence=token_list[:i+1]
    sequences.append(n_gram_sequence)



In [10]:
max_sequnce_length=max([len(seq) for seq in sequences ])
padded_sequences=np.array(pad_sequences(sequences,maxlen=max_sequnce_length,padding='pre'))

In [11]:
input_sequences,labels=padded_sequences[:,:-1],padded_sequences[:,-1]

In [12]:
one_hot_labels=tf.keras.utils.to_categorical(labels,num_classes=total_words)

In [13]:
model=tf.keras.models.Sequential([
                            tf.keras.layers.Embedding(total_words,64,input_length=max_sequnce_length-1),
                            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20)),
                            tf.keras.layers.Dense(total_words,activation='softmax')
])

In [14]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 19, 64)            598144    
_________________________________________________________________
bidirectional (Bidirectional (None, 40)                13600     
_________________________________________________________________
dense (Dense)                (None, 9346)              383186    
Total params: 994,930
Trainable params: 994,930
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history=model.fit(input_sequences,one_hot_labels,epochs=200,verbose=1)

Epoch 1/200
5760/5760 [==============================] - 73s 13ms/step - loss: 6.2535 - accuracy: 0.0573
Epoch 2/200
5760/5760 [==============================] - 75s 13ms/step - loss: 5.6391 - accuracy: 0.1049
Epoch 3/200
5760/5760 [==============================] - 76s 13ms/step - loss: 5.3318 - accuracy: 0.1295
Epoch 4/200
5760/5760 [==============================] - 75s 13ms/step - loss: 5.1088 - accuracy: 0.1478
Epoch 5/200
5760/5760 [==============================] - 75s 13ms/step - loss: 4.9385 - accuracy: 0.1622
Epoch 6/200
5760/5760 [==============================] - 78s 14ms/step - loss: 4.8044 - accuracy: 0.1731
Epoch 7/200
5760/5760 [==============================] - 77s 13ms/step - loss: 4.6973 - accuracy: 0.1837
Epoch 8/200
5760/5760 [==============================] - 77s 13ms/step - loss: 4.6082 - accuracy: 0.1929
Epoch 9/200
5760/5760 [==============================] - 75s 13ms/step - loss: 4.5326 - accuracy: 0.2017
Epoch 10/200
5760/5760 [==============================]